[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nijl/ddc-tutorial2025/blob/main/notebooks/tutorial_04_gephi/tutorial_04_gephi.ipynb)

# 可視化4：ネットワーク分析（Gephi Lite）

**ネットワークで見る文学界の構造**

RAWGraphsでは「量」や「流れ」を見てきましたが、ここでは**「つながり」**を可視化します。

## 使用ツール
- **Gephi Lite**: https://gephi.org/gephi-lite/
  - Webブラウザで動作（インストール不要）
  - ネットワーク分析・可視化ツール

## データの見方
- **ノード（点）**: 著者、ジャンル
- **エッジ（線）**: 「著者がそのジャンルの本を書いた」という関係

## 目的
- 「多才な作家」と「専門作家」の違いを見る
- ジャンル同士の近さ（同じ著者が書いているジャンルは近い）を見る

---
## 1. データの読み込み

In [ ]:
import pandas as pd
import networkx as nx

# Google Colabでファイルをアップロードする場合
try:
    from google.colab import files
    print("ndl_sru_for_rawgraphs.csv をアップロードしてください")
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    df = pd.read_csv(filename)
except ImportError:
    # ローカル環境の場合
    df = pd.read_csv('../../data/ndl_sru_for_rawgraphs.csv')

print(f"データ件数: {len(df)}件")
df.head()

---
## 2. エッジリスト（関係性リスト）の作成

ネットワークを描画するための**エッジリスト**を作成します。

- `Source`（出発点）: 著者
- `Target`（到着点）: ジャンル

これにより「著者Aがジャンルαの本を書いた」という関係を表現します。

In [ ]:
# エッジリストの作成（著者 → ジャンル）
edges_df = df[['creator_normalized', 'genre_major']].copy()
edges_df.columns = ['Source', 'Target']

# 「不明」と「その他」を除外（見やすいネットワークのため）
edges_df = edges_df[
    (edges_df['Source'] != '不明') & 
    (edges_df['Source'] != 'その他') &
    (edges_df['Target'] != '不明')
]

print(f"エッジ数: {len(edges_df)}件")
edges_df.head(10)

### エッジの重み付け

同じ著者×ジャンルの組み合わせが複数ある場合、その回数を「重み（Weight）」として集計します。

In [ ]:
# 重み付きエッジリストの作成
edges_weighted = edges_df.groupby(['Source', 'Target']).size().reset_index(name='Weight')

print(f"ユニークなエッジ数: {len(edges_weighted)}件")
edges_weighted.sort_values('Weight', ascending=False).head(10)

---
## 3. NetworkXでグラフを作成

Gephi Liteで読み込めるGEXF形式でグラフを構築します。

In [ ]:
# NetworkXグラフを作成
G = nx.Graph()

# ノードリストの作成
authors = edges_df['Source'].unique()
genres = edges_df['Target'].unique()

# 著者ノードを追加
for author in authors:
    G.add_node(f"著者:{author}", label=author, node_type="著者")

# ジャンルノードを追加
for genre in genres:
    G.add_node(f"ジャンル:{genre}", label=genre, node_type="ジャンル")

# エッジを追加
for _, row in edges_weighted.iterrows():
    G.add_edge(
        f"著者:{row['Source']}",
        f"ジャンル:{row['Target']}",
        weight=int(row['Weight'])
    )

print(f"ノード数: {G.number_of_nodes()}")
print(f"  - 著者: {len(authors)}")
print(f"  - ジャンル: {len(genres)}")
print(f"エッジ数: {G.number_of_edges()}")

---
## 4. GEXFファイルの出力

Gephi Liteで読み込めるGEXF形式でエクスポートします。

In [ ]:
# GEXFファイルとして保存
gexf_filename = 'author_genre_network.gexf'
nx.write_gexf(G, gexf_filename)

print(f"GEXFファイルを作成しました: {gexf_filename}")

In [ ]:
# Google Colabの場合、ファイルをダウンロード
try:
    from google.colab import files
    print("ファイルをダウンロードしています...")
    files.download(gexf_filename)
except ImportError:
    print(f"ローカル環境: カレントディレクトリに保存しました")

---
## 5. Gephi Lite での操作手順

### 5.1 データの読み込み
1. [Gephi Lite](https://gephi.org/gephi-lite/) にアクセス
2. 「**Open**」→「**Open a file from your computer**」をクリック
3. ダウンロードした `author_genre_network.gexf` を選択
4. 自動的にネットワークが描画されます

### 5.2 レイアウトの適用
1. 左側パネルの「**Layout**」タブを開く
2. 「**ForceAtlas 2**」を選択
3. **Start（再生ボタン）**を押す
4. ノードが関係性に基づいて移動し、クラスタが形成される様子を観察
5. 適度に収束したら **Stop** で停止

### 5.3 見た目の調整
1. 左側パネルの「**Appearance**」タブを開く
2. **Nodes** → **Color** → **Partition** → **node_type** を選択
   - 著者とジャンルで色分け
3. **Nodes** → **Size** → **Ranking** → **Degree** を選択
   - ハブが大きく表示される

### 5.4 読み取りポイント

#### ハブ（中心人物）の発見
- 多くのジャンルと結びついている著者は誰か？
- 例：曲亭馬琴、柳亭種彦など → 文学界のキーパーソン

#### クラスタ（集団）の発見
- 「和歌・国学」のグループと「戯作・滑稽本」のグループは分かれているか？
- → 江戸の「真面目な本」と「娯楽本」の距離感がわかる

---
## 6. 統計情報の確認

In [ ]:
# 著者ごとのジャンル数（多才な作家を発見）
author_genres = edges_df.groupby('Source')['Target'].nunique().sort_values(ascending=False)

print("=== 多ジャンル執筆著者 TOP10 ===")
print(author_genres.head(10))

In [ ]:
# ジャンルごとの著者数
genre_authors = edges_df.groupby('Target')['Source'].nunique().sort_values(ascending=False)

print("=== ジャンル別著者数 ===")
print(genre_authors)

In [ ]:
# 著者×ジャンルの組み合わせ上位（よく書かれた組み合わせ）
print("=== 著者×ジャンル 執筆数 TOP15 ===")
print(edges_weighted.sort_values('Weight', ascending=False).head(15))

---
## まとめ

### 作成したファイル
| ファイル名 | 形式 | 用途 |
|-----------|------|------|
| `author_genre_network.gexf` | GEXF | Gephi Liteで読み込み |

### 可視化ツールの使い分け
- **RAWGraphs**: 分布・構造・時間変化を見る
- **Gephi Lite**: 関係性・ネットワーク構造を見る

データを「ネットワーク」として捉え直すことで、当時の文壇の人間関係や構造が見えてきます。